In [18]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

park = Customer('Park', 100)
print(park)

class LineItem:
  def __init__(self, product, quantity, price):
    self.product = product
    self.quantity = quantity
    self.price = price

  def total(self):
    return self.price * self.quantity

class Order:
  def __init__(self, customer, cart, promotion=None):
    self.customer = customer
    self.cart = list(cart)
    self.promotion = promotion

  def total(self):
    if not hasattr(self, '__total'):
      self.__total = sum(item.total() for item in self.cart)
    return self.__total

  def due(self):
    if self.promotion is None:
      discount = 0
    else:
      discount = self.promotion.discount(self)
    return self.total() - discount

  def __repr__(self):
    fmt = '<Order total: {:.2f} due: {:.2f}>'
    return fmt.format(self.total(), self.due())

class Promotion(ABC):

  @abstractmethod
  def discount(self, order):
    pass

class FidelityPromo(Promotion):
  def discount(self, order):
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion):
  def discount(self, order):
    discount = 0
    for item in order.cart:
      if item.quantity >= 20:
        discount += item.total() * .1
    return discount

class LargeOrderPromo(Promotion):
  def discount(self, order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
      return order.total() * .07
    return 0

Customer(name='Park', fidelity=100)


In [19]:
doe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

print(Order(doe, cart, FidelityPromo()))
print(Order(ann, cart, FidelityPromo()))

banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]
print(Order(doe, banana_cart, BulkItemPromo()))

long_order = [LineItem(str(item_code), 1, 1.0)
              for item_code in range(10)]
print(Order(doe, long_order, LargeOrderPromo()))
print(Order(doe, cart, LargeOrderPromo()))

<Order total: 42.00 due: 42.00>
<Order total: 42.00 due: 39.90>
<Order total: 30.00 due: 28.50>
<Order total: 10.00 due: 9.30>
<Order total: 42.00 due: 42.00>


In [21]:
class Order:
  def __init__(self, customer, cart, promotion=None):
    self.customer = customer
    self.cart = list(cart)
    self.promotion = promotion

  def total(self):
    if not hasattr(self, '__total'):
      self.__total = sum(item.total() for item in self.cart)
    return self.__total

  def due(self):
    if self.promotion is None:
      discount = 0
    else:
      discount = self.promotion(self)
    return self.total() - discount

  def __repr__(self):
    fmt = '<Order total: {:.2f} due: {:.2f}>'
    return fmt.format(self.total(), self.due())

def fidelity_promo(order):
  return order.total() * .05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * .1
  return discount

def large_order_promo(order):
  distinct_items = {item.product for item in order.cart}
  if len(distinct_items) >= 10:
    return order.total() * .07
  return 0

print(Order(doe, cart, fidelity_promo))
print(Order(ann, cart, fidelity_promo))
print(Order(doe, banana_cart, bulk_item_promo))
print(Order(doe, long_order, large_order_promo))

<Order total: 42.00 due: 42.00>
<Order total: 42.00 due: 39.90>
<Order total: 30.00 due: 28.50>
<Order total: 10.00 due: 9.30>


In [24]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
  return max(promo(order) for promo in promos)

In [25]:
print(Order(doe, long_order, best_promo))
print(Order(doe, banana_cart, best_promo))
print(Order(ann, cart, best_promo))

print(globals().keys())

print(globals()['best_promo'])

<Order total: 10.00 due: 9.30>
<Order total: 30.00 due: 28.50>
<Order total: 42.00 due: 39.90>
dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'ABC', 'abstractmethod', 'namedtuple', 'Customer', 'park', '_i2', 'LineItem', 'Order', 'Promotion', 'FidelityPromo', 'BulkItemPromo', '_i3', 'doe', 'ann', 'cart', '_i4', '_4', '_i5', '_i6', '_6', '_i7', '_7', '_i8', '_8', '_i9', '_9', '_i10', '_10', '_i11', '_11', '_i12', '_12', '_i13', '_i14', '_14', '_i15', '_i16', '_i17', 'banana_cart', 'long_order', '_i18', 'LargeOrderPromo', '_i19', '_i20', 'fidelity_promo', 'bulk_item_promo', 'large_order_promo', '_i21', '_i22', 'promos', 'best_promo', '_i23', '_i24', '_i25'])
<function best_promo at 0x7bd3886e2020>


In [26]:
promos = [globals()[name] for name in globals()
          if name.endswith('_promo')
          and name != 'best_promo']

print(promos)

[<function fidelity_promo at 0x7bd39922dc60>, <function bulk_item_promo at 0x7bd388826ca0>, <function large_order_promo at 0x7bd388827100>]
